In [1]:
import pandas as pd
import numpy as np

In [2]:
# read dts.xlsx and dts2.xlsx
dts = pd.read_excel('dts.xlsx')
dts2 = pd.read_excel('dts2.xlsx')
# hanya ambil full_text
dts = dts['full_text']
dts2 = dts2['full_text']

# gabungkan dts dan dts2
df = pd.concat([dts, dts2], ignore_index=True)
df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 116 entries, 0 to 115
Series name: full_text
Non-Null Count  Dtype 
--------------  ----- 
116 non-null    object
dtypes: object(1)
memory usage: 1.0+ KB


In [3]:
# df to dataframe
df = pd.DataFrame(df)
df.head()

,full_text
0,Padahal buka aja kalo emang yakin pemerintah u...
1,Saya perhatikan hampir semua akun akun besar d...
2,"Biar adil, mahasiswa KKN di Papua\nTNI ambil S..."
3,Kalian judulnya jurnalis tapi misi sebenarnya ...
4,THIS emang TNI itu dikasih belajar HAM gak sih...


In [4]:
# lakukan praproses dalam bahasa indonesia
import string
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

# remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# remove number

# define stemming function
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(text)

def remove_number(text):
    return re.sub(r'\d+', '', text)

# remove whitespace
def remove_whitespace(text):
    return text.strip()

# remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])

nltk.download('stopwords')
df['full_text'] = df['full_text'].apply(stemming)

[nltk_data] Downloading package stopwords to C:\Users\GLOBAL
[nltk_data]     KOMPUTER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['full_text'] = df['full_text'].apply(remove_punctuation)
df['full_text'] = df['full_text'].apply(remove_number)
df['full_text'] = df['full_text'].apply(remove_whitespace)
df['full_text'] = df['full_text'].apply(remove_stopwords)
df['full_text'].head()


0    buka aja kalo emang perintah udah bener mah ka...
1    perhati akun akun indonesia instagram x kompak...
2    biar adil mahasiswa kkn papua tni ambil simak ...
3    judul jurnalis misi separatis ya negara yg kas...
4    this emang tni kasih ajar ham gak sih angkat t...
Name: full_text, dtype: object

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

pretrained= "mdhugol/indonesia-bert-sentiment-classification"
model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)   
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

label_index = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}



c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
def predict_label(text):
    return sentiment_analysis(text)[0]

df['sentimen'] = df['full_text'].apply(predict_label)
df['sentimen'] = df['sentimen'].apply(lambda x: label_index[x['label']])
df[['full_text', 'sentimen']].head()


,full_text,sentimen
0,buka aja kalo emang pemerintah udah bener mah ...,negative
1,perhatikan akun akun indonesia instagram X kom...,neutral
2,Biar adil mahasiswa KKN Papua TNI ambil Simak ...,negative
3,judulnya jurnalis misi separatis Ya negara yg ...,negative
4,THIS emang TNI dikasih belajar HAM gak sih ang...,negative


In [12]:
processed = df[['full_text', 'sentimen']]
processed['sentimen'].value_counts()

sentimen
neutral     55
negative    52
positive     9
Name: count, dtype: int64

In [13]:
df.head()

,full_text,sentimen
0,buka aja kalo emang pemerintah udah bener mah ...,negative
1,perhatikan akun akun indonesia instagram X kom...,neutral
2,Biar adil mahasiswa KKN Papua TNI ambil Simak ...,negative
3,judulnya jurnalis misi separatis Ya negara yg ...,negative
4,THIS emang TNI dikasih belajar HAM gak sih ang...,negative


In [14]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


<p>Some words would be broken into alphabets and a double hash(##) was added in front of w and p. This technique used by BERT is known as <b>Subword tokenization </b>. The “##” symbol is used to indicate that a token is part of a larger word.</p>

In [15]:
test_input_1='TNI bagus ayo amankan bumi indonesia dari serangan musuh'
test_input_2='Kalau main tentara tentaraan yang bener dong jangan asal main'
inputs=[test_input_1,test_input_2]

tokenizer.tokenize(inputs)

['tni',
 'bagus',
 'ayo',
 'aman',
 '##kan',
 'bumi',
 'indonesia',
 'dari',
 'serangan',
 'musuh',
 'kalau',
 'main',
 'tentara',
 'tentara',
 '##an',
 'yang',
 'bener',
 'dong',
 'jangan',
 'asal',
 'main']

In [16]:
output=tokenizer(inputs,padding=True,truncation=True,max_length=128)
print(output)

{'input_ids': [[3, 3136, 4839, 15124, 4022, 1493, 2859, 1718, 1542, 3416, 5084, 4, 0], [3, 2280, 5539, 3682, 3682, 1476, 1497, 13927, 6773, 3132, 2785, 5539, 4]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [17]:
tokenizer.decode(output['input_ids'][0])

'[CLS] tni bagus ayo amankan bumi indonesia dari serangan musuh [SEP] [PAD]'

In [18]:
def preprocess_function(examples):
  return tokenizer(examples["text"],padding=True,truncation=True,) 

In [19]:
# apply preprocess function to our text data
encoded_data = df['full_text'].apply(lambda x: tokenizer(x,padding=True,truncation=True,max_length=128))
encoded_data.head()

0    [input_ids, token_type_ids, attention_mask]
1    [input_ids, token_type_ids, attention_mask]
2    [input_ids, token_type_ids, attention_mask]
3    [input_ids, token_type_ids, attention_mask]
4    [input_ids, token_type_ids, attention_mask]
Name: full_text, dtype: object

In [20]:
# make encoded data as a df with its full text
encoded_data_df = pd.DataFrame(encoded_data.tolist())
encoded_data_df['full_text'] = processed['full_text']
encoded_data_df['sentimen'] = processed['sentimen']
encoded_data_df.head()

,attention_mask,input_ids,token_type_ids,full_text,sentimen
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 9012, 6380, 7288, 17673, 1990, 9988, 13927...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",buka aja kalo emang pemerintah udah bener mah ...,negative
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 10245, 7408, 7408, 1718, 23292, 1517, 2230...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",perhatikan akun akun indonesia instagram X kom...,neutral
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 9421, 6187, 2928, 13174, 4464, 3136, 8009,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Biar adil mahasiswa KKN Papua TNI ambil Simak ...,negative
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 21934, 8406, 4746, 14958, 3316, 1806, 3798...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",judulnya jurnalis misi separatis Ya negara yg ...,negative
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 7185, 17673, 3136, 21463, 3109, 2972, 7552...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",THIS emang TNI dikasih belajar HAM gak sih ang...,negative


In [21]:
encoded_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   attention_mask  116 non-null    object
 1   input_ids       116 non-null    object
 2   token_type_ids  116 non-null    object
 3   full_text       116 non-null    object
 4   sentimen        116 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


In [22]:
max_length = max(
    max(len(x) for x in encoded_data_df['attention_mask']),
    max(len(x) for x in encoded_data_df['input_ids']),
    max(len(x) for x in encoded_data_df['token_type_ids'])
)

In [24]:
from sklearn.preprocessing import LabelEncoder

def pad_array(arr, max_length):
    return arr + [0] * (max_length - len(arr))

X = []
for i in range(len(encoded_data_df)):
    attention_mask = pad_array(encoded_data_df['attention_mask'][i], max_length)
    input_ids = pad_array(encoded_data_df['input_ids'][i], max_length)
    token_type_ids = pad_array(encoded_data_df['token_type_ids'][i], max_length)
    
    features = np.concatenate([attention_mask, input_ids, token_type_ids])
    X.append(features)
X = np.array(X)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentimen'])


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Make predictions
y_pred = svm.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.5833333333333334
Classification Report:
              precision    recall  f1-score   support

    negative       0.60      0.60      0.60        10
     neutral       0.67      0.57      0.62        14
    positive       0.00      0.00      0.00         0

    accuracy                           0.58        24
   macro avg       0.42      0.39      0.41        24
weighted avg       0.64      0.58      0.61        24



c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\GLOBAL KOMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera